# Speed Testing: Show Homes - Analysis

Analysis done for show home speed testing in March 2022.

In [1]:
%load_ext autoreload
%autoreload 2

import json
import requests
import yaml
from time import sleep 
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict 

from keplergl import KeplerGl

from scipy.spatial.distance import cdist
from geopy.distance import distance as geodist 

from asf_core_data import load_preprocessed_epc_data
from asf_core_data.getters.epc.epc_data import load_preprocessed_epc_data
from asf_core_data.pipeline.preprocessing import feature_engineering

from show_homes import PROJECT_DIR
from show_homes.pipeline import geo_utils


In [ ]:
LOCAL_DATA_DIR = '/Users/juliasuter/Documents/ASF_data'
IMG_OUTPUT_PATH = PROJECT_DIR / 'outputs/figures'

MAPS_OUTPUT_PATH = PROJECT_DIR / "outputs/maps/"
MAPS_CONFIG_PATH = PROJECT_DIR / "outputs/maps/config/"
MAPS_CONFIG_FILE = MAPS_CONFIG_PATH / "show_home_network.txt"
OUTPUT_DATA_DIR = PROJECT_DIR / "outputs/data/"

DIST_FILE = PROJECT_DIR / 'outputs/data/distances_and_durations.csv'
DIST_FILE_NEW = PROJECT_DIR / 'outputs/data/distances_and_durations_updated.csv'

### Analysis driving and air distances and travel times 

Show driving distances, air distances and travel times for potential show home network.

In [ ]:
df = pd.read_csv(DIST_FILE)
df.head()

## 

In [ ]:
flat_any = df.loc[(df['Property Type'] == 'Flat') & (df['HP Property'] == 'any')]
semi_detached_any = df.loc[(df['Property Type'] == 'Semi-detached house') & (df['HP Property'] == 'any')]
detached_any = df.loc[(df['Property Type'] == 'Detached House') & (df['HP Property'] == 'any')]
terraced_any = df.loc[(df['Property Type'] == 'Terraced House') & (df['HP Property'] == 'any')]
all_any = df.loc[(df['Property Type'] == 'All') & (df['HP Property'] == 'any')]

flat_similar = df.loc[(df['Property Type'] == 'Flat') & (df['HP Property'] == 'similar')]
semi_detached_similar = df.loc[(df['Property Type'] == 'Semi-detached house') & (df['HP Property'] == 'similar')]
detached_similar = df.loc[(df['Property Type'] == 'Detached House') & (df['HP Property'] == 'similar')]
terraced_similar = df.loc[(df['Property Type'] == 'Terraced House') & (df['HP Property'] == 'similar')]
all_similar = df.loc[(df['Property Type'] == 'All') & (df['HP Property'] == 'similar')]

In [ ]:
print('All')
print('-----')
print('Samples:', all_any.shape[0])
print('Distance (mean):', round(all_any['Distance'].mean(),1))
print('Duration (mean):', round(all_any['Duration'].mean(),1))
print()
print('Distance (mean):', round(all_similar['Distance'].mean(),1))
print('Duration (mean):', round(all_similar['Duration'].mean(),1))
print()
print('Flat')
print('-----')
print('Samples:', flat_any.shape[0])
print('Distance (mean):', round(flat_any['Distance'].mean(),1))
print('Duration (mean):', round(flat_any['Duration'].mean(),1))
print()
print('Samples:', flat_similar.shape[0])
print('Distance (mean):', round(flat_similar['Distance'].mean(),1))
print('Duration (mean):', round(flat_similar['Duration'].mean(),1))
print()
print('Detached House')
print('-----')
print('Samples:', detached_any.shape[0])
print('Distance (mean):', round(detached_any['Distance'].mean(),1))
print('Duration (mean):', round(detached_any['Duration'].mean(),1))
print()
print(detached_similar.shape[0])
print('Distance (median):', round(detached_similar['Distance'].mean(),1))
print('Duration (median):', round(detached_similar['Duration'].mean(),1))
print()
print('Semi-detached House')
print('-----')
print('Samples:', semi_detached_any.shape[0])
print('Distance (mean):', round(semi_detached_any['Distance'].mean(),1))
print('Duration (mean):', round(semi_detached_any['Duration'].mean(),1))
print()
print('Samples:', semi_detached_similar.shape[0])
print('Distance (mean):', round(semi_detached_similar['Distance'].mean(),1))
print('Duration (mean):', round(semi_detached_similar['Duration'].mean(),1))
print()
print('Terraced House')
print('-----')
print('Samples:', terraced_any.shape[0])
print('Distance (mean):', round(terraced_any['Distance'].mean(),1))
print('Duration (mean):', round(terraced_any['Duration'].mean(),1))
print()
print(terraced_similar.shape[0])
print('Distance (mean):', round(terraced_similar['Distance'].mean(),1))
print('Duration (mean):', round(terraced_similar['Duration'].mean(),1))
print()




In [ ]:
print('All')
print('-----')
print('Samples:', all_any.shape[0])
print('Distance (median):', round(all_any['Distance'].median(),1))
print('Duration (median):', round(all_any['Duration'].median(),1))
print()
print('Distance (median):',round(all_similar['Distance'].median(),1))
print('Duration (median):', round(all_similar['Duration'].median(),1))
print()
print('Flat')
print('-----')
print('Samples:', flat_any.shape[0])
print('Distance (median):',round(flat_any['Distance'].median(),1))
print('Duration (median):', round(flat_any['Duration'].median(),1))
print()
print('Samples:', flat_similar.shape[0])
print('Distance (median):',round(flat_similar['Distance'].median(),1))
print('Duration (median):', round(flat_similar['Duration'].median(),1))
print()
print('Detached House')
print('-----')
print('Samples:', detached_any.shape[0])
print('Distance (median):',round(detached_any['Distance'].median(),1))
print('Duration (median):', round(detached_any['Duration'].median(),1))
print()
print(detached_similar.shape[0])
print('Distance (median):',round(detached_similar['Distance'].median(),1))
print('Duration (median):', round(detached_similar['Duration'].median(),1))
print()
print('Semi-detached House')
print('-----')
print('Samples:', semi_detached_any.shape[0])
print('Distance (median):',round(semi_detached_any['Distance'].median(),1))
print('Duration (median):', round(semi_detached_any['Duration'].median(),1))
print()
print('Samples:', semi_detached_similar.shape[0])
print('Distance (median):',round(semi_detached_similar['Distance'].median(),1))
print('Duration (median):', round(semi_detached_similar['Duration'].median(),1))
print()
print('Terraced House')
print('-----')
print('Samples:', terraced_any.shape[0])
print('Distance (median):',round(terraced_any['Distance'].median(),1))
print('Duration (median):', round(terraced_any['Duration'].median(),1))
print()
print('Samples:', terraced_similar.shape[0])
print('Distance (median):', round(terraced_similar['Distance'].median(),1))
print('Duration (median):', round(terraced_similar['Duration'].median(),1))
print()

In [ ]:
distances = {'Flat' : flat_any,
             'Semi-detached House': semi_detached_any,
             'Detached House': detached_any,
             'Terraced House' : terraced_any,
             'All' : all_any}

distances_similar = {'Flat' : flat_similar,
             'Semi-detached House': semi_detached_similar,
             'Detached House': detached_similar,
             'Terraced House' : terraced_similar,
             'All' : all_any}

cond_labels = ['All', 'Flat',  'Terraced House','Detached House','Semi-detached House']


In [ ]:
for label in cond_labels:
    sns.kdeplot(data=distances[label]['Distance'], fill=True, common_norm=False, alpha=0.1)
plt.legend(cond_labels)
plt.xlabel('Distance [km]')
plt.title('Distance to nearest HP property')
plt.xlim(-5,75)
plt.savefig(IMG_OUTPUT_PATH / 'Distance to nearest HP property.png', dpi=300)
plt.show()

for label in cond_labels:
    sns.kdeplot(data=distances_similar[label]['Distance'], fill=True, common_norm=False, alpha=0.1)
plt.legend(cond_labels)
plt.xlabel('Distance [km]')
plt.title('Distance to nearest similar HP property')
plt.xlim(-5,75)
plt.savefig(IMG_OUTPUT_PATH / 'Distance to nearest similar HP property.png', dpi=300)
plt.show()

In [ ]:
for label in cond_labels:
    sns.kdeplot(data=distances[label]['Duration'], fill=True, common_norm=False, alpha=0.1)
plt.legend(cond_labels)
plt.xlabel('Travel Time [min]')
plt.title('Travel Time to nearest HP property')
plt.xlim(-5,75)
plt.savefig(IMG_OUTPUT_PATH / 'Travel Time to nearest HP property.png', dpi=300)
plt.show()

for label in cond_labels:
    sns.kdeplot(data=distances_similar[label]['Duration'], fill=True, common_norm=False, alpha=0.1)
plt.legend(cond_labels)
plt.xlabel('Travel Time [min]')
plt.title('Travel Time to nearest similar HP property')
plt.xlim(-5,75)
plt.savefig(IMG_OUTPUT_PATH / 'Travel Time to nearest similar HP property.png', dpi=300)
plt.show()

In [ ]:
for label in cond_labels:
    dist_matrix = np.load(OUTPUT_DATA_DIR / Path(label+'_any.npy'))
    dist_matrix = np.min(dist_matrix, axis=1)
    sns.kdeplot(data=dist_matrix, fill=True, common_norm=False, alpha=0.1)
plt.legend(cond_labels)
plt.xlabel('Distance [km]')
plt.title('Air distance to nearest HP property')
plt.xlim(-5,75)
plt.savefig(IMG_OUTPUT_PATH / 'Air distance to nearest HP property.png', dpi=300)
plt.show()

for label in cond_labels:
    dist_matrix_similar = np.load(OUTPUT_DATA_DIR / Path(label+'_similar.npy'))
    dist_matrix_similar = np.min(dist_matrix_similar, axis=1)
    sns.kdeplot(data=dist_matrix_similar, fill=True, common_norm=False, alpha=0.1)
    
plt.legend(cond_labels)
plt.xlabel('Distance [km]')
plt.title('Air distance to nearest similar HP property')
plt.xlim(-5,75)
plt.savefig(IMG_OUTPUT_PATH / 'Air distance to nearest similar HP property.png', dpi=300)
plt.show()

### Map showing distance between to closest similar property with heat pump

In [ ]:

with open(MAPS_CONFIG_FILE, "r") as infile:
    config = yaml.load(infile.read(), Loader=yaml.FullLoader)

flat_similar = flat_similar.round({'Distance':1, 'Duration':1})
semi_detached_similar = semi_detached_similar.round({'Distance':1, 'Duration':1})
detached_similar = detached_similar.round({'Distance':1, 'Duration':1})
terraced_similar = terraced_similar.round({'Distance':1, 'Duration':1})

flat_similar.rename(columns={'Distance':'Distance [in km]'}, inplace=True)
semi_detached_similar.rename(columns={'Distance':'Distance [in km]'}, inplace=True)
detached_similar.rename(columns={'Distance':'Distance [in km]'}, inplace=True)
terraced_similar.rename(columns={'Distance':'Distance [in km]'}, inplace=True)


distance = KeplerGl(height=500, config=config)

distance.add_data(
    data=flat_similar[["Latitude 1", "Longitude 1", "Latitude 2", "Longitude 2", 'Distance [in km]', 'Duration']], name="Flat")

distance.add_data(
    data=semi_detached_similar[["Latitude 1", "Longitude 1", "Latitude 2", "Longitude 2", 'Distance [in km]', 'Duration']], name="Semi-detached House")

distance.add_data(
    data=detached_similar[["Latitude 1", "Longitude 1", "Latitude 2", "Longitude 2", 'Distance [in km]', 'Duration']], name="Detached House")

distance.add_data(
    data=terraced_similar[["Latitude 1", "Longitude 1", "Latitude 2", "Longitude 2", 'Distance [in km]', 'Duration']], name="Terraced House")

#distance.add_data(
  #  data=all_similar[["Latitude 1", "Longitude 1", "Latitude 2", "Longitude 2", 'Distance', 'Duration']], name="All")

distance


In [ ]:
with open(MAPS_CONFIG_FILE, "w") as outfile:
    outfile.write(str(distance.config))

distance.save_to_html(file_name=MAPS_OUTPUT_PATH / "show_homes_distances_similar.html")

### Computing the driving distance for HP and no-HP properties

Load the EPC data, split into different property types and randomly sample 10 000 samples for properies with heat pumps and properties without. Then compute the driving distance and travel time. 

In [ ]:
EPC_FEAT_SELECTION =  [
    "ADDRESS1",
    "ADDRESS2",
    "POSTCODE",
    "LOCAL_AUTHORITY_LABEL",
    "CURRENT_ENERGY_RATING",
    "POTENTIAL_ENERGY_RATING",
    "TENURE",
    "HP_INSTALLED",
    "INSPECTION_DATE",
    "BUILT_FORM",
    "PROPERTY_TYPE",
    "CONSTRUCTION_AGE_BAND",
    "TRANSACTION_TYPE",
    "TOTAL_FLOOR_AREA",
    "HP_TYPE"
  ]

epc_df = load_preprocessed_epc_data(data_path=LOCAL_DATA_DIR, 
            version="preprocessed_dedupl", usecols=EPC_FEAT_SELECTION
        )

epc_df = feature_engineering.get_postcode_coordinates(epc_df)
epc_df['COORDS'] = list(zip(epc_df['LATITUDE'],epc_df['LONGITUDE']))

In [ ]:
terraced = ["Enclosed Mid-Terrace", "Enclosed End-Terrace", "End-Terrace", "Mid-Terrace"]
no_cond = ~epc_df['LATITUDE'].isna()
flat = epc_df['PROPERTY_TYPE'] == 'Flat'
terraced_house = ((epc_df['PROPERTY_TYPE'] == 'House') & (epc_df['BUILT_FORM'].isin(terraced)))
detached_house = ((epc_df['PROPERTY_TYPE'] == 'House') & (epc_df['BUILT_FORM'] == 'Detached'))               
semi_house = ((epc_df['PROPERTY_TYPE'] == 'House') & (epc_df['BUILT_FORM'] == 'Semi-Detached'))

 
cond_labels = ['Flat', 'Semi-detached house', 'Detached House', 'Terraced House', 'All']

# Original settings
# N_NO_HP_SAMPLES = 10000
# N_HP_SAMPLES = 10000

# For testing/refactoring
N_NO_HP_SAMPLES = 100
N_HP_SAMPLES = 10

dist_matrices = {}
dist_matrices_similar = {}

with open(DIST_FILE_NEW, 'w') as outfile:

    for i, conds in enumerate([flat, semi_house, detached_house, terraced_house, no_cond]):

        label = cond_labels[i]

        epc_df_sampled = epc_df.loc[~epc_df['HP_INSTALLED']].sample(frac=1, random_state=42)[:N_NO_HP_SAMPLES]
        hp_sampled = epc_df.loc[epc_df['HP_INSTALLED']].sample(frac=1, random_state=42)[:N_HP_SAMPLES]

        print(label)
        print('-----')

        epc_df_sampled = epc_df_sampled.loc[conds]
        print('# Properties:', epc_df_sampled.shape[0])

        prop_coords = epc_df_sampled[['LATITUDE', 'LONGITUDE']].to_numpy()
        hp_prop_coords = hp_sampled[['LATITUDE', 'LONGITUDE']].to_numpy()

        dist_matrix = cdist(prop_coords, hp_prop_coords, lambda u, v: geodist(u, v).km) # choose unit

        hp_prop_coords_similar_type = hp_sampled.loc[conds][['LATITUDE', 'LONGITUDE']].to_numpy()
        print('# Similar HP Properties', hp_prop_coords_similar_type.shape[0])

        dist_matrix_similar = cdist(prop_coords, hp_prop_coords_similar_type, lambda u, v: geodist(u, v).km) # choose unit

        print('Mean distance:', np.min(dist_matrix, axis=1).mean())
        print('Mean distance, similar property:', np.min(dist_matrix_similar, axis=1).mean())

        dist_matrices[label] = dist_matrix
        dist_matrices_similar[label] = dist_matrix_similar

        np.save(PROJECT_DIR / f'outputs/data/{label}_any.npy', dist_matrix)
        np.save(PROJECT_DIR / f'outputs/data/{label}_similar.npy', dist_matrix_similar)

        for dist_label, matrix in zip(['any', 'similar'],[dist_matrix, dist_matrix_similar]):

            distances = []
            durations = []

            failed = []
            ind = np.argmin(matrix, axis=1)
            
            hp_property_coords = hp_prop_coords_similar_type if dist_label == 'similar' else  hp_prop_coords

            for i, coord_pair in enumerate(zip(prop_coords, hp_property_coords[ind])):
                                    
                if label == 'All' and dist_label == 'similar':
                    continue

                coords_1 = (coord_pair[0][0],coord_pair[0][1])
                coords_2 = (coord_pair[1][0],coord_pair[1][1])

                dist_duration = geo_utils.get_travel_distance_and_duration(coords_1, coords_2)
                if dist_duration is None:
                    failed.append(i)
                    continue
                
                distance, duration = dist_duration
                distances.append(distance)
                durations.append(duration)
                
                out = [str(i), label,dist_label,str(coords_1[0]),str(coords_1[1]),str(coords_2[0]),str(coords_2[1]),str(distance),str(duration)]
                out = ','.join(out)
                
                outfile.write(out)
                outfile.write('\n')

                sleep(1.5)

            print('Distances:',np.array(distances).mean())
            print('Durations:',np.array(durations).mean())
            print('Failed:', failed)

KeyboardInterrupt: 